In [2]:
from __future__ import print_function, division, absolute_import
import os,sys
import numpy as np
sys.path.insert(0, "C:\\Users\\Matt\\Desktop\\torihunter-master\\")
from torihunter import *
from scipy.linalg import inv, block_diag
import time
from joblib import Parallel, delayed

In [3]:
meths = ['nelder-mead','powell','cg','bfgs','newton-cg', 'l-bfgs-b',
                'tnc','cobyla','slsqp','trust-constr','dogleg','trust-ncg',
                'trust-krylov','trust-exact']

In [4]:
filename = 'C:\\Users\\Matt\\Desktop\\gudorf\\KS\\python\\data_and_figures\\blocks\\none\\data\\none_001_012.h5'
x = read_h5(filename, cls=Orbit, statetype='field')
x = rediscretize(x, newN=128, newM=64)
np.random.seed(50)
# x = Orbit(T=129, L=65)
# x.plot()
x.state += 0.1*np.random.randn(*(x.state.shape))
# x.plot()

In [ ]:
with Parallel(n_jobs=-2) as parallel:
    results = parallel(delayed(converge)(x, method=method, options={'maxiter':100, 'disp':True}) for method in meths)

In [4]:
t0 = time.time()
xnm = converge(x, method='nelder-mead', verbose=False, options={'maxiter':1000, 'disp':True})
tf = time.time()-t0

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:500: RuntimeWarning: Method nelder-mead does not use gradient information (jac).
  RuntimeWarning)


KeyboardInterrupt: 

In [ ]:
t0 = time.time()

xp = converge(x, method='powell', verbose=False, options={'maxiter':1000, 'disp':True})
print(time.time()-t)

In [ ]:
t0 = time.time()

xcg = converge(x, method='cg', verbose=False, options={'maxiter':1000, 'disp':True})
print(time.time()-t)

In [ ]:
t0 = time.time()
xbfgs = converge(x, method='bfgs', verbose=False, options={'maxiter':1000, 'disp':True})


In [ ]:
t0 = time.time()
xncg = converge(x, method='newton-cg', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xlbfgsb = converge(x, method='l-bfgs-b', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xtnc = converge(x, method='tnc', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xcobyla = converge(x, method='cobyla', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xslsqp = converge(x, method='slsqp', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xtrust_constr = converge(x, method='trust-constr', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xdogleg = converge(x, method='dogleg', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xtrust_ncg = converge(x, method='trust-ncg', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xtrust_krylov = converge(x, method='trust-krylov', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
t0 = time.time()
xtrust_exact  =converge(x, method='trust-exact', verbose=False, options={'maxiter':1000, 'disp':True})

In [ ]:
xtuple = converge(x, method='l-bfgs-b', verbose=False)

In [ ]:
x.rmatvec(x.spatiotemporal_mapping()).state_vector().ravel()

In [ ]:
t = time.time()
xtuple = converge(x, method='adj', verbose=False)
x = xtuple.Orbit
test0 = converge(x, method='lstsq', verbose=False)
print('\n')
print('time for lstsq', time.time()-t, 'residual', test0.Orbit.residual())
t = time.time()
test1 = converge(x, method='lsqr', verbose=False)
print('time for lsqr', time.time()-t, 'residual', test1.Orbit.residual())
t = time.time()
test2 = converge(x, method='lsmr', verbose=False)
print('time for lsmr', time.time()-t, 'residual', test2.Orbit.residual())
# t = time.time()

test0.Orbit.plot()
test1.Orbit.plot()
test2.Orbit.plot()
result = test0

if __name__=='__main__':
    sys.exit(main())

In [ ]:
A = x.jac()

In [ ]:
x.matvec(x).state.ravel()-np.dot(A, x.state_vector()).ravel() 

In [ ]:
P = x.preconditioner()
Pr = x.preconditioner(side='right')

In [ ]:
P.shape, Pr.shape

In [ ]:
qk_matrix = x.elementwise_dx()
wj_matrix = x.elementwise_dt()
pdiag = 1 / (np.abs(wj_matrix) + qk_matrix**2 + qk_matrix**4)
pdiag = np.diag(pdiag.ravel())
parameter_block = []
# parameter_block.append(1/(x.T))
# parameter_block.append(1/(x.L**4))
parameter_block.append(1)
parameter_block.append(1)
parameter_block = np.array(parameter_block)
P = block_diag(pdiag, np.diag(parameter_block.ravel()))

In [ ]:
np.linalg.cond(np.multiply(P, A))

In [ ]:
np.linalg.cond(np.multiply(A, Pr))

In [ ]:
np.linalg.cond(np.dot(P[:-2,:-2], A))

In [ ]:
np.linalg.cond(np.dot(A, P))

In [ ]:
np.linalg.cond(A)